In [1]:
import pyw_hnswlib as hnswlib
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import re
import string

import os

In [3]:
index_file_path = "/home/installer/ps/hnsw_backup/my_index"


# Load index from binary file
index = hnswlib.Index(space='l2', dim=768)
index.load_index(index_file_path)

In [4]:

# device = "cpu"
# # from sentence_transformers import SentenceTransformer

# model_salesken = SentenceTransformer("salesken/similarity-eng-hin_latin")


In [2]:
# Load the model from a pickle file
with open('model_salesken.pkl', 'rb') as f:
    model_salesken = pickle.load(f)

In [5]:

# Save the model to a pickle file
with open('model_salesken.pkl', 'wb') as f:
    pickle.dump(model_salesken, f)

In [3]:
r = pd.read_csv('data/chunk0.csv',  header = None, nrows = 1, low_memory=False)

In [7]:
# Load the pandas dataframe
df1 = pd.read_csv('data/chunk0.csv',  header = None, low_memory=False)
df2 = pd.read_csv('data/chunk1.csv', header = None, low_memory=False)
df = pd.concat([df1, df2])




In [8]:

df = df[[0,21]]
df.columns = ['reg_no', 'subject_content']

In [9]:
# df.subject_content.to_csv("tmp.csv")

In [10]:

# Function to split text into lines with maximum words_per_line words per line
def split_text_to_lines(text, words_per_line=20):
    words = text.split()
    lines = []
    line = []
    for word in words:
        line.append(word)
        if len(line) == words_per_line:
            lines.append(' '.join(line))
            line = []
    if line:
        lines.append(' '.join(line))
    return '\n'.join(lines)



In [11]:
with open('final_wording.pkl', 'rb') as f:
    final_wording = pickle.load(f)

In [12]:

def clean_text(text):
    """
    Preprocess textual data by performing the following steps:
    1. Remove punctuation
    2. Convert text to lowercase
    3. Remove digits and special characters
    4. Remove extra whitespaces
    """
    if not isinstance(text, (str, bytes)):
        return text
    
    # remove links
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove digits and special characters
    text = re.sub(r'\d+', '', text)
    
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Correct Roman words using the predefined dictionary
    words = text.split()
    for i in range(len(words)):
        if words[i] in final_wording:
            words[i] = final_wording[words[i]]
    text = ' '.join(words)
    
    return text

In [16]:

# Query the index
query = 'tax frauds in many states'
query_embedding = model_salesken.encode(clean_text(query))
labels, distances = index.knn_query(query_embedding, k=25)

out  = df[df.reg_no.isin(labels[0])]
# print("here")
# print(out.head(20))

# Apply the split_text_to_lines function to all columns in the DataFrame
for col in out.columns:
    out[col] = out[col].apply(split_text_to_lines)

# # Call the function to create the PDF report
# create_pdf_report(out, 'report.pdf')
    
out.to_csv(query+".csv", index=False, lineterminator='\r\n\n')
    
    

/tmp/ipykernel_1255574/824235642.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out[col] = out[col].apply(split_text_to_lines)
/tmp/ipykernel_1255574/824235642.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out[col] = out[col].apply(split_text_to_lines)
